In [1]:
import pysnooper
import pandas as pd
from skimage import exposure
import numpy as np
import cv2
import random
import copy

In [2]:
input_csv = pd.read_csv("/home/extension/kaggle/APTOS_2019_Blindness_Detection/train.csv")

In [3]:
input_csv.describe()

,diagnosis
count,3662.000000
mean,1.126980
std,1.298409
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,4.000000


In [4]:
input_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 2 columns):
id_code      3662 non-null object
diagnosis    3662 non-null int64
dtypes: int64(1), object(1)
memory usage: 57.3+ KB


In [5]:
# @pysnooper.snoop()
def light_more(image):
    flag = random.uniform(1, 1.45)
    return exposure.adjust_gamma(image, flag)

In [6]:
# @pysnooper.snoop()
def light_less(image):
    flag = random.uniform(0.56, 1)
    return exposure.adjust_gamma(image, flag)

In [7]:
# @pysnooper.snoop()
def filp_image(image):
    if random.random() < 0.5:
        horizon = True
    else:
        horizon = False
            
    if horizon: #水平翻转
        image =  cv2.flip(image, 1)
    else:
        image = cv2.flip(image, 0)
        
    return image

In [8]:
function_list = [light_more, light_less, filp_image]

In [9]:
max_sample = max([input_csv.loc[input_csv["diagnosis"] == tmp]["diagnosis"].count() for tmp in input_csv["diagnosis"].unique()])

In [10]:
print(max_sample)

1805


In [11]:
img_root_path = "/home/extension/kaggle/APTOS_2019_Blindness_Detection/train_images/"

In [12]:
new_dict = {}

for tmp_class in input_csv["diagnosis"].unique():
    tmp_collection = input_csv.loc[input_csv["diagnosis"] == tmp_class]
    
    tmp_count = tmp_collection["diagnosis"].count()
    
    if tmp_count >= max_sample:
        continue

    index_collection = tmp_collection.index
    
    tmp_id_code = []
    tmp_diagnosis = []
    
    for tmp_index in range(max_sample - tmp_count):
        img_name = tmp_collection.loc[index_collection[tmp_index%tmp_count]]["id_code"]
        img_path = img_root_path + img_name + ".png"
        
        tmp_name = str(tmp_class) + "_" + str(tmp_index) + "_" + img_name

        tmp_img = random.choice(function_list)(cv2.imread(img_path))
        cv2.imwrite(img_root_path + tmp_name + ".png", tmp_img)
        
        tmp_id_code.append(tmp_name)
        tmp_diagnosis.append(tmp_class)
        
    new_dict[tmp_class] = {"id_code": copy.deepcopy(tmp_id_code), "diagnosis": copy.deepcopy(tmp_diagnosis)}

In [13]:
for tmp_class in new_dict:
    new_df = pd.DataFrame({
        "id_code": new_dict[tmp_class]["id_code"],
        "diagnosis": new_dict[tmp_class]["diagnosis"]
    })
    
    input_csv = input_csv.append(new_df)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [14]:
input_csv.to_csv("new_submission.csv", index=False)

In [15]:
tmp_df = pd.read_csv("new_submission.csv")

In [18]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 2 columns):
diagnosis    9025 non-null int64
id_code      9025 non-null object
dtypes: int64(1), object(1)
memory usage: 141.1+ KB


In [21]:
tmp_df = tmp_df.drop(index=2094)

In [23]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9024 entries, 0 to 9024
Data columns (total 2 columns):
diagnosis    9024 non-null int64
id_code      9024 non-null object
dtypes: int64(1), object(1)
memory usage: 211.5+ KB


In [24]:
tmp_df.to_csv("final.csv", index=False)

In [15]:
final_train = pd.read_csv("final.csv")

In [16]:
final_train["id_code"] = final_train["id_code"].map(lambda x:str(x))

In [17]:
final_train["diagnosis"] = final_train["diagnosis"].map(lambda x:str(x))

In [18]:
final_train["id_code"].str.cat(final_train["diagnosis"], "_")

0              000c1434d8d7_2
1              001639a390f0_4
2              0024cdab0c1e_1
3              002c21358ce6_0
4              005b95c28852_0
5              0083ee8054ee_4
6              0097f532ac9f_0
7              00a8624548a9_2
8              00b74780d31d_2
9              00cb6555d108_1
10             00cc2b75cddd_0
11             00e4ddff966a_2
12             00f6c1be5a33_0
13             0104b032c141_3
14             0124dffecf29_1
15             0125fbd2e791_0
16             012a242ac6ff_2
17             014508ccb9cb_0
18             0151781fe50b_0
19             0161338f53cc_2
20             0180bfa26c0b_2
21             0182152c50de_0
22             01b3aed3ed4c_1
23             01c7808d901d_2
24             01d9477b1171_0
25             01eb826f6467_2
26             01f7bb8be950_0
27             0212dd31f623_0
28             022f820027b8_0
29             0231642cf1c2_0
                ...          
8994    4_1480_0318598cfd16_4
8995    4_1481_034cb07a550f_4
8996    4_

In [7]:
final_train

,diagnosis,id_code
0,2,0 000c1434d8d7\n1 00...
1,4,0 000c1434d8d7\n1 00...
2,1,0 000c1434d8d7\n1 00...
3,0,0 000c1434d8d7\n1 00...
4,0,0 000c1434d8d7\n1 00...
5,4,0 000c1434d8d7\n1 00...
6,0,0 000c1434d8d7\n1 00...
7,2,0 000c1434d8d7\n1 00...
8,2,0 000c1434d8d7\n1 00...
9,1,0 000c1434d8d7\n1 00...
